<Strong> Boston Housing Regression Problem</Strong>

In [59]:
from keras.datasets import boston_housing
from tensorflow.keras import optimizers
import numpy as np
from keras import models
from keras import layers

In [62]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

In [68]:
print(train_data.shape)
test_data.shape

(404, 13)


(102, 13)

<Strong> Each one of the examples has 13 features such as crime rate , temperature </Strong>

In [69]:
#Avg is 10,000 - 50,000
train_targets
test_targets

array([ 7.2, 18.8, 19. , 27. , 22.2, 24.5, 31.2, 22.9, 20.5, 23.2, 18.6,
       14.5, 17.8, 50. , 20.8, 24.3, 24.2, 19.8, 19.1, 22.7, 12. , 10.2,
       20. , 18.5, 20.9, 23. , 27.5, 30.1,  9.5, 22. , 21.2, 14.1, 33.1,
       23.4, 20.1,  7.4, 15.4, 23.8, 20.1, 24.5, 33. , 28.4, 14.1, 46.7,
       32.5, 29.6, 28.4, 19.8, 20.2, 25. , 35.4, 20.3,  9.7, 14.5, 34.9,
       26.6,  7.2, 50. , 32.4, 21.6, 29.8, 13.1, 27.5, 21.2, 23.1, 21.9,
       13. , 23.2,  8.1,  5.6, 21.7, 29.6, 19.6,  7. , 26.4, 18.9, 20.9,
       28.1, 35.4, 10.2, 24.3, 43.1, 17.6, 15.4, 16.2, 27.1, 21.4, 21.5,
       22.4, 25. , 16.6, 18.6, 22. , 42.8, 35.1, 21.5, 36. , 21.9, 24.1,
       50. , 26.7, 25. ])

<Strong> Feature-wise normalization , else it will make learning more difficult </Strong>

<Strong> axis=0 (columns) , subtract mean / standard deviation </Strong>

<Strong> s.t = How spread out the numbers are. Mean , (each no. - mean)^2 , mean of all them , square root that </Strong>

<Strong> Use training data for normalizing the test data. </Strong>

In [70]:
mean = train_data.mean(axis=0)
train_data -= mean

std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

In [71]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                           input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

<Strong>  Applying an activation function would constrain the range the out- put can take </Strong>

<Strong> mse - difference between predicted and actual </Strong>

<Strong> K-Fold for if we have a small number of training exampes . the validation scores might change a lot depending on which data points you chose to use for validation and which you chose for training.</Strong>

In [79]:
k=4
num_val_samples = len(train_data) // k 
num_epochs = 100
all_scores = []

for i in range(k):
    
    print('processing fold #', i)
    
    #Setting the k-fold for validation , first round validation is the first fold
    #0 - 101 , 101 - 202 , 202 - 303 , 303 - 404
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples] 
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    
    #Skipping the k-fold validation fold .
    # upto 0 and 101 - 
    partial_train_data = np.concatenate( [train_data[:i * num_val_samples] , train_data[(i + 1) * num_val_samples:]], axis=0)
    partial_train_targets = np.concatenate([train_targets[:i * num_val_samples] , train_targets[(i + 1) * num_val_samples:]], axis=0)
    
    #Instantiating the model with different training folds for the loop
    model = build_model()
    history = model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=1, verbose=0)
    
    #evaluating on each validation fold after training on the other folds
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=1)
    all_scores.append(val_mae)

processing fold # 0
4/4 [==============================] - 0s 1ms/step - loss: 10.3395 - mae: 2.3813
processing fold # 1
4/4 [==============================] - 0s 1ms/step - loss: 10.2430 - mae: 2.3189
processing fold # 2
4/4 [==============================] - 0s 1ms/step - loss: 17.4403 - mae: 2.8781
processing fold # 3
4/4 [==============================] - 0s 2ms/step - loss: 12.5452 - mae: 2.5178


In [80]:
all_scores

[2.3812813758850098, 2.3188724517822266, 2.8781075477600098, 2.517775058746338]

In [81]:
np.mean(all_scores)

2.524009108543396

[[3]
 [4]]


(102, 13)